## Add sparklines in jupyter notebook

In [50]:
from datetime import timedelta, date
from itertools import chain
from time import sleep
import requests
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
import base64
from IPython.display import display, HTML

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [51]:
def sparkline(data, figsize=(4, 0.25), **kwags):
    """
    Returns a HTML image tag containing a base64 encoded sparkline style plot
    """
    data = list(data)
    
    fig, ax = plt.subplots(1, 1, figsize=figsize, **kwags)
    ax.plot(data)
    for k,v in ax.spines.items():
        v.set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])    

    plt.plot(len(data) - 1, data[len(data) - 1], 'r.')

    ax.fill_between(range(len(data)), data, len(data)*[min(data)], alpha=0.1)
    
    img = BytesIO()
    plt.savefig(img)
    img.seek(0)
    plt.close()
    return '<img src="data:image/png;base64,{}"/>'.format(base64.b64encode(img.read()).decode())

## Random data Example

In [52]:
n = 100

data = [
    ('rand',        np.random.rand(n)), 
    ('randn',       np.random.randn(n)), 
    ('beta',        np.random.beta(2, 1, size=n)),
    ('binomial',    np.random.binomial(3.4, 0.22, size=n)),
    ('exponential', np.random.exponential(size=n)),
    ('geometric',   np.random.geometric(0.5, size=n)),
    ('laplace',     np.random.laplace(size=n))
]
spark = pd.DataFrame(data, columns=['func', 'data'])
spark

,func,data
0,rand,"[0.15476947905203742, 0.8713051614159419, 0.36..."
1,randn,"[0.2898378659874084, -0.16068225782126905, 1.0..."
2,beta,"[0.8009300469659278, 0.4773160622654932, 0.859..."
3,binomial,"[0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 0, 1, 0, 2, 0, ..."
4,exponential,"[0.7634610070508059, 0.36284302414910835, 3.46..."
5,geometric,"[1, 1, 1, 1, 2, 1, 1, 2, 5, 2, 2, 4, 1, 3, 1, ..."
6,laplace,"[-0.4808760960327676, 2.181444352306794, 1.093..."


In [53]:
# map the sparkline function over the data column
# and store back in the column sparklines
spark['sparklines'] = spark.data.map(sparkline)
spark = spark[['func','sparklines']]
# _repr_html_ escapes HTML so manually handle the rendering
HTML(spark.to_html(escape=False))

,func,sparklines
0,rand,
1,randn,
2,beta,
3,binomial,
4,exponential,
5,geometric,
6,laplace,


## COVID-19 Sparklines

In [54]:
Confirmed_filepath = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

In [55]:
# Import covid 19 data from Johnhopkins Repository
def Import_COVIDData(filepath):
        data = filepath
        data = pd.read_csv(data)
        del data['Province/State']
        del data['Lat']
        del data['Long']
        data.rename(columns={'Country/Region':'Country'},inplace = True)
        return data

In [56]:
def transform_COVIDData(data):
        data = pd.melt(
        frame = data,
        id_vars = 'Country',
        var_name = 'Date',
        value_name = 'Count'
    )
        return data

In [57]:
ConfirmedCases = Import_COVIDData(Confirmed_filepath)
ConfirmedCases = transform_COVIDData(ConfirmedCases)

In [58]:
ConfirmedCases

,Country,Date,Count
0,Afghanistan,1/22/20,0
1,Albania,1/22/20,0
2,Algeria,1/22/20,0
3,Andorra,1/22/20,0
4,Angola,1/22/20,0
...,...,...,...
131445,Vietnam,5/13/21,3740
131446,West Bank and Gaza,5/13/21,303270
131447,Yemen,5/13/21,6507
131448,Zambia,5/13/21,92262


In [59]:
ConfirmedCases = ConfirmedCases.groupby(['Country','Date'])['Count'].sum().to_frame()
ConfirmedCases = ConfirmedCases.reset_index()
ConfirmedCases['Date'] = pd.to_datetime(ConfirmedCases['Date'])
ConfirmedCases.set_index('Date',inplace= True)
ConfirmedCases

,Country,Count
Date,,
2021-01-01,Afghanistan,51526
2021-01-10,Afghanistan,53489
2021-01-11,Afghanistan,53538
2021-01-12,Afghanistan,53584
2021-01-13,Afghanistan,53584
...,...,...
2020-09-05,Zimbabwe,6837
2020-09-06,Zimbabwe,6837
2020-09-07,Zimbabwe,7298


In [60]:
# Function to retrive no of days from the most recent date
def Coviddays(data,days):
    dt = list(set(data.index))
    dt.sort()
    date =dt[-days:]
    TotaldataConfirmed =(data.loc[ConfirmedCases.index[data.index.isin(date)]])
    return TotaldataConfirmed

In [61]:
TotaldataConfirmed = Coviddays(ConfirmedCases,10)
TotaldataConfirmed = TotaldataConfirmed.drop_duplicates()

In [62]:
TotaldataConfirmed

,Country,Count
Date,,
2021-05-10,Afghanistan,62063
2021-05-10,Albania,131753
2021-05-10,Algeria,124288
2021-05-10,Andorra,13429
2021-05-10,Angola,28875
...,...,...
2021-05-09,Vietnam,3412
2021-05-09,West Bank and Gaza,301751
2021-05-09,Yemen,6482


In [63]:
TotaldataConfirmed = TotaldataConfirmed.reset_index()

In [64]:
dt = TotaldataConfirmed[['Country','Count']]

In [65]:
data = dt.groupby('Country')['Count'].apply(list)
spark = pd.DataFrame(data, columns=['Count'])
spark = spark.reset_index()

In [66]:
spark

,Country,Count
0,Afghanistan,"[62063, 62403, 62718, 63045, 60563, 60797, 611..."
1,Albania,"[131753, 131803, 131845, 131890, 131327, 13141..."
2,Algeria,"[124288, 124483, 124682, 124889, 122999, 12327..."
3,Andorra,"[13429, 13447, 13470, 13316, 13340, 13363, 133..."
4,Angola,"[28875, 29146, 29405, 29695, 27284, 27529, 279..."
...,...,...
187,Vietnam,"[3489, 3571, 3658, 3740, 2995, 3030, 3090, 313..."
188,West Bank and Gaza,"[302249, 302777, 303270, 298921, 299736, 30038..."
189,Yemen,"[6485, 6492, 6498, 6507, 6390, 6414, 6426, 644..."
190,Zambia,"[92112, 92152, 92211, 92262, 91804, 91849, 919..."


In [67]:
TotaldataConfirmed[TotaldataConfirmed['Country'] == 'US']

,Date,Country,Count
178,2021-05-10,US,32745255
336,2021-05-11,US,32778906
494,2021-05-12,US,32814784
640,2021-05-13,US,32852871
816,2021-05-04,US,32512934
972,2021-05-05,US,32557669
1128,2021-05-06,US,32605183
1281,2021-05-07,US,32652472
1419,2021-05-08,US,32686965
1552,2021-05-09,US,32708357


## Covid 19 Trend for last 10 days 

In [68]:
spark['sparklines'] = spark.Count.map(sparkline)
spark = spark[['Country','sparklines']]
# _repr_html_ escapes HTML so manually handle the rendering
HTML(spark.to_html(escape=False))

,Country,sparklines
0,Afghanistan,
1,Albania,
2,Algeria,
3,Andorra,
4,Angola,
5,Antigua and Barbuda,
6,Argentina,
7,Armenia,
8,Australia,
9,Austria,
